<a href="https://colab.research.google.com/github/Alekhyo/Decision-Tree/blob/master/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math

## 1. You have to work on the play tennis(https://www.kaggle.com/fredericobreno/play-tennis) dataset and manually find out on which column the first split will take place.

In [0]:
df=pd.read_csv('play_tennis.csv')
df

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [0]:
def calculate_entropy(val):     #Function to calculate entropy defined by the formula: -Py.log(Py)-Pn.log(Pn)
    try:
        val=float(val)
        return round((-val*math.log(val,2)-(1-val)*math.log(1-val,2)),2)
    except:
        return 0.0

In [0]:
def divide(val1,val2):         #Function to divide two numbers
    try:
        return (round(val1/val2,2))
    except:
        return 0.0

In [0]:
parent=calculate_entropy(df[df['play']=='Yes'].shape[0]/df.shape[0])  #Calculating entropy of Parent

In [0]:
def find_IG(col_name):
    IG=parent
    categories=df[col_name].unique()
    no_of_category=len(categories)
    weighted_average=[]
    entropy_of_children=[]
    
    for name in categories:
        weighted_average.append(divide(df.loc[df[col_name]==name].shape[0],df.shape[0])) #Find weighted average of every children
        entropy_of_children.append(divide((df.loc[(df[col_name]==name) & (df['play']=='Yes')].shape[0]),(df.loc[df[col_name]==name].shape[0])))
        
    for i in range(0,no_of_category):
        entropy_of_children[i]=calculate_entropy(entropy_of_children[i]) #Find entropy of every children
    
    for i in range(0,no_of_category):
        IG=IG-(weighted_average[i]*entropy_of_children[i])
        #Calculate IG=entropy(PARENT)-weighted_average*entropy(CHILDREN)
    
    return(round(IG,2))

In [0]:
IG_dict={}

In [0]:
for i in df.columns[1:-1]:  #Storing IG of every column except 'DAY' and 'PLAY'
    IG_dict[i]=find_IG(i)

In [0]:
print(" Hence the splitting the dataset along the feature {} results in the largest information gain and we should use this feature for our root node.".format(max(IG_dict, key=IG_dict.get))) 

 Hence the splitting the dataset along the feature outlook results in the largest information gain and we should use this feature for our root node.


## OUTLOOK column has highest IG

##  2. You have to work on the iris dataset and manually find out the splitting criteria for Petal Length column

In [0]:
iris=pd.read_csv('iris.csv')
iris.drop(columns={'Id','SepalLengthCm','SepalWidthCm','PetalWidthCm'},inplace=True)
iris.rename(columns={'PetalLengthCm':'PL'},inplace=True)

In [0]:
iris=iris.sort_values('PL')

In [0]:
def calculate_multi_entropy(array):    #Function to calculate entropy using: -P1log(P1)-P2log(P2)-P3log(P3)
    try:
        res=0
        for i in range(0,len(array)):
            res=res-array[i]*math.log(array[i],2)
        return res
    except:
        return 0

In [0]:
category=[]
for species in iris['Species'].unique():
    category.append(iris[iris['Species']==species].shape[0]/iris.shape[0])

In [0]:
parent=calculate_multi_entropy(category) #Entropy of Parent

In [0]:
def find_IG_numerical(df,split_column,value):   #Function to calculate entropy of Numerical Column
    value=float(value)
    entropy_of_left_child=[]
    weighted_of_left_child=0
    
    entropy_of_right_child=[]
    weighted_of_right_child=0
    
    df_left=iris[iris[split_column]<=value]    #Dataframe where PL<=threshold
    df_right=iris[iris[split_column]>value]     #Dataframe where PL>threshold
    
    for species in iris['Species'].unique():
        entropy_of_left_child.append(divide(df_left[df_left['Species']==species].shape[0],df_left.shape[0]))
        weighted_of_left_child=divide(df_left.shape[0],df.shape[0])     #Weight of child1
        
        entropy_of_right_child.append(divide(df_right[df_right['Species']==species].shape[0],df_right.shape[0]))
        weighted_of_right_child=divide(df_right.shape[0],df.shape[0])     #Weight of child2
    
    
    entropy_of_left_child=calculate_multi_entropy(entropy_of_left_child)  #Entropy of child1
    entropy_of_right_child=calculate_multi_entropy(entropy_of_right_child)  #Entropy of child2
    
    weight_total=weighted_of_left_child*entropy_of_left_child+weighted_of_right_child*entropy_of_right_child 
    #Weighted_average*Entropy(Children)
    
    return (parent-weight_total)      #IG=Entropy(PARENT)-Weighted_average*Entropy(Children)

In [0]:
IG=[]                                         #Finding the IG of every value in PL(Petal_length) column
for i in iris['PL'].values:
    IG.append(find_IG_numerical(iris,'PL',i))
IG=np.array(IG)                              

In [0]:
iris.iloc[np.argmax(IG)]                     #Finding the threshold value for ideal splitting

PL                 1.9
Species    Iris-setosa
Name: 24, dtype: object

## Petal_length=1.9 has the highest IG